In [3]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import re
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
import getpass
import smtplib, ssl
from email.message import EmailMessage
from email import message
from google.oauth2 import service_account
from googleapiclient.discovery import build
import streamlit as st

In [4]:
def extract_text_from_URL(URL):
    response=requests.get(URL)
    soup=BeautifulSoup(response.text,'html.parser')
    paragraphs=soup.find_all('p')
    text_list=[]
    for p in paragraphs:
        p=p.get_text().strip() #removing extra spaces
        p=re.sub(r'\[\d+\]','',p)
        if p!='\n':
            text_list.append(str(p))
    text=' '.join(text_list)
    return text

In [5]:
def summary_text(text):
    model_name='gpt-3.5-turbo'
    text_splitter=CharacterTextSplitter.from_tiktoken_encoder(model_name=model_name)
    texts=text_splitter.split_text(text)
    docs=[Document(page_content=t) for t in texts]
    llm=ChatOpenAI(temperature=0,model_name=model_name)
    prompt_template=("""
    Write a concise summary of the following text:
    '{text}'
    While giving output consider following points:-
        - Output text should be in the form of paragraphs
        - Remove any extra spaces or unwanted special characters from the output text
        - If input contains any promotional sentences like '5% off or Shop Now!, Be the first to grab the offer, Buy one get one free, etc'. Don't consider such sentences while giving the output text.
    """)
    prompt=ChatPromptTemplate([
        ('system','You are a content writer expert'),
        ('human',prompt_template)
    ])
    # prompt=PromptTemplate(template=prompt_template,input_variables=['text'])
    chain=prompt | llm
    summary=chain.invoke(docs)
    return summary

In [6]:
def export_to_word(text,filename):
    with open(filename,'w') as f:
        f.write(text)

In [7]:
def get_email_input():
    # sender_email=str(input("Sender Email Address: "))
    reciever_email=[]
    while True:
        reciever=str(input("Reciever email address (enter 0 if you dont want to add another email): "))
        if reciever=='0' and len(reciever_email)==0:
            print('Enter atleast one reciever email address')
            continue
        elif reciever=='0' and len(reciever_email)>0:
            break
        elif reciever=='sagaragarwal2052@gmail.com':
            print("reciever email can't be same as sender email")
            continue
        elif reciever in reciever_email:
            print("Email already entered")
            continue    
        else:
            reciever_email.append(reciever)
    
    # password=getpass.getpass('Enter Password: ')
    return reciever_email
    

In [8]:
def send_mail(reciever_email_list,text):
    #Construct Email
    message=EmailMessage()
    message['From']='sagaragarwal2052@gmail.com'
    message['To']=', '.join(reciever_email_list) if len(reciever_email_list)>1 else reciever_email_list[0]
    message['Subject']='Summary text'
    message.set_content(text)

    port=465
    # context=ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com',port) as server:
        server.login('sagaragarwal2052@gmail.com','gcpx gbxu wzbt ldnt')
        server.send_message(message)
    print('Email sent')

In [ ]:
if __name__=='__main__':
    load_dotenv()
    url=str(input('Enter URL:'))
    print('Extracting content...')
    raw_text=extract_text_from_URL(url)

    print('Summarizing...')
    summary=summary_text(raw_text)
    print(summary.content)
    print('Exporting to word file...')
    export_to_word(summary.content,'summary.docx')
    print('Sending Email...')
    reciever_emails=get_email_input()
    send_mail(reciever_emails,summary.content)